In [1]:
import pandas as pd
import json
import os

folder_path = "Sample Data"
file_name = "products.csv"
file_path = os.path.join(folder_path, file_name)

with open(file_path, "r", encoding="utf-8") as f:
    df = pd.read_csv(f)
print(df)
# Convert column names dynamically
df.columns = (
    df.columns.str.strip()  # Remove leading/trailing spaces
    .str.lower()            # Convert to lowercase
    .str.replace(" ", "_")  # Replace spaces with underscores
    .str.replace(r"[^\w\s]", "", regex=True)  # Remove special characters
)

    Product ID                Product Name     Category Sub Category    Price  \
0            1                      Laptop  Electronics       Mobile  2084.66   
1            2                  Smartphone  Electronics        Audio  2948.17   
2            3                      Tablet  Electronics       Gaming   349.22   
3            4                  Smartwatch  Electronics       Gaming  1803.71   
4            5            Wireless Earbuds  Electronics       Gaming   616.41   
5            6              Gaming Console  Electronics       Gaming   624.51   
6            7                     Monitor  Electronics        Audio  2307.92   
7            8         Mechanical Keyboard  Electronics        Audio   459.38   
8            9         External Hard Drive  Electronics  Accessories  2110.69   
9           10                    Smart TV  Electronics  Accessories   611.33   
10          11           Bluetooth Speaker  Electronics    Computing  2041.78   
11          12              

In [3]:
import snowflake.connector
import os
import requests
PASSWORD = os.getenv('SNOWSQL_PWD')
print(PASSWORD)

5eWxWv4EvyCkhkY


In [9]:
try:
    ctx = snowflake.connector.connect(
        user='ABHINAVSHARMA2002',
        password=PASSWORD,
        account='qraojwa-yg67137'
    )
    cs = ctx.cursor()
    try:
        cs.execute("CREATE WAREHOUSE IF NOT EXISTS task_1_warehouse_mg")
        cs.execute("CREATE DATABASE IF NOT EXISTS testdb_mg")
        cs.execute("USE DATABASE testdb_mg")
        cs.execute("CREATE SCHEMA IF NOT EXISTS task_1_mg")
        cs.execute("USE WAREHOUSE task_1_warehouse_mg")
        cs.execute("USE SCHEMA task_1_mg")
        runQueries(cs,df)
    except Exception as e:
        print(f"Error: {e}") 
    finally:
        cs.close()
        ctx.close()
except Exception as e:
    print(f"Error: {e}")

In [7]:
def runQueries(cs,df):    
    # Rename specific columns
    #df["customer_id"] = df["customer_id"].astype(str)
    df = df.astype(str).apply(lambda col: col.map(lambda x: None if pd.isna(x) or x == "" or str(x).strip().lower() == "nan" else x))

    # Generate CREATE TABLE statement dynamically
    table_name = "article_table_pandas_5"
    columns = ", ".join([f'{col} STRING' for col in df.columns])
    create_table_sql = f'CREATE OR REPLACE TABLE {table_name} ({columns})'
    cs.execute(create_table_sql)

    # Insert data into Snowflake
    for _, row in df.iterrows():
        placeholders = ", ".join(["%s"] * len(row))
        insert_sql = f'INSERT INTO {table_name} ({", ".join(df.columns)}) VALUES ({placeholders})'
        cs.execute(insert_sql, tuple(row))